In [1]:
from snowflake.snowpark import Session
import pandas as pd
import requests
from snowflake.snowpark.functions import sproc

In [2]:
#Connecting to snowflake
connection_parameters = {
"account": "ld77469.uae-north.azure",
"user": "GOKS98",
"password": "March20311",
"role": "ACCOUNTADMIN",
"warehouse": "SNOWFLAKE_WAREHOUSE",
"database": "OIL_AND_GAS",
"schema": "CONFORMED"
 }

session = Session.builder.configs(connection_parameters).create()

In [3]:
session.add_packages('pandas', 'requests', 'snowflake-snowpark-python')

@sproc(session=session, is_permanent=True, stage_location='@MY_STAGE', name='sp_weather_join', replace=True)
def api_weather(session: Session) -> str:
    #Creating a Dataframe
    df_table = session.table("SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.WAREHOUSE")

    #Converting to pandas df to make manipulation easier
    df_pd = df_table.to_pandas()

    #Getting weather data from API for every zipcode
    json_list = []
    for i in df_pd['W_ZIP']:
        response = requests.get(f"http://api.openweathermap.org/geo/1.0/zip?zip={i},US&appid=b3effbd4d36024a4b9735d33830e7ca3")
        data = response.json()
        json_list.append(data)
    json_list

    #Creating a new df with weather info and zip code
    zip_code = df_pd['W_ZIP']
    data = list(zip(zip_code, json_list))
    weather_df = pd.DataFrame(data=data, columns=['W_ZIP', 'WEATHER_JSON'])
    
    #Joining weather nad warehouse tables
    snowflake_df = df_pd.merge(weather_df, how='inner', on='W_ZIP')

    snowflake_df_1 = session.create_dataframe(snowflake_df)
    #Writing table to snowflake
    snowflake_df_1.write.mode("overwrite").save_as_table("w_warehouse_weather")

    return 'Success!!'
    

The version of package snowflake-snowpark-python in the local environment is 0.12.0, which does not fit the criteria for the requirement snowflake-snowpark-python. Your UDF might not work when the package version is different between the server and your local environment


In [6]:
#Creating a Dataframe
df_table = session.table("SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.WAREHOUSE")

#Converting to pandas df to make manipulation easier
df_pd = df_table.to_pandas()

#Getting weather data from API for every zipcode
json_list = []
for i in df_pd['W_ZIP']:
    response = requests.get(f"http://api.openweathermap.org/geo/1.0/zip?zip={i},US&appid=b3effbd4d36024a4b9735d33830e7ca3")
    data = response.json()
    json_list.append(data)
json_list

#Creating a new df with weather info and zip code
zip_code = df_pd['W_ZIP']
data = list(zip(zip_code, json_list))
weather_df = pd.DataFrame(data=data, columns=['W_ZIP', 'WEATHER_JSON'])

#Joining weather nad warehouse tables
snowflake_df = df_pd.merge(weather_df, how='inner', on='W_ZIP')

snowflake_df_1 = session.create_dataframe(snowflake_df)
#Writing table to snowflake
snowflake_df_1.write.mode("overwrite").save_as_table("w_warehouse_weather")

create_temp_table is deprecated. We still respect this parameter when it is True but please consider using `table_type="temporary"` instead.
